### Given data,
### 1. Job locations: The locations of the four sites, with site 1 representing the home office and sites 2, 3, and 4 representing remote job sites.  

### 2. Equipment trailer cost: The cost of relocating the equipment trailer between the job sites, which is defined as d(j, k) = 300 for k ≠ j.  

### 3. Trailer usage cost: The cost of using the trailer for each job, which is defined as 100 if the work is at site k > 1 and trailer is at site j ≠ k with j > 1, 50 if j = k and j > 1, and 200 if the work is at k > 1 and the trailer is at j = 1 (home office).  

### 4. Assume the discount factor of 0.95. 

### 5. Transition matrix: The transition matrix between job locations, which provides the probabilities of moving between the different job sites.  

|     | 1    | 2    | 3    | 4    |
|-----|------|------|------|------|
| **1** | 0.1  | 0.3  | 0.3  | 0.3  |
| **2** | 0.0  | 0.5  | 0.5  | 0.0  |
| **3** | 0.0  | 0.0  | 0.8  | 0.2  |
| **4** | 0.4  | 0.0  | 0.0  | 0.6  |


### Method 1 : Obtain optimal policy and optimal value function by value iteration and calculate lower and upper bounds 

In [35]:
import numpy as np

def calculate_cost(j, k, u):
    if k == 1 and u == j:
        return 0
    elif k > 1 and u == j == 1:
        return 200
    elif k == u == j > 1:
        return 50
    elif k > 1 and u != j and u != k:
        return 100
    elif k == 1 and u != j:
        return 300
    elif k > 1 and u == 1 and j != 1:
        return 500
    elif k > 1 and u == k and j != k:
        return 350
    elif k > 1 and (u == 1 or u == k) and j != u:
        return 400
    else:
        return 0  # or a value that makes sense in your context

def value_iteration(P, discount_factor, tolerance=1e-6):
    num_states = P.shape[0]
    
    # Initialize the value functions
    V = np.zeros((num_states, num_states))
    V_lo = np.zeros((num_states, num_states))
    V_up = np.zeros((num_states, num_states))
    
    while True:
        V_prev = V.copy()
        V_delta = np.zeros((num_states, num_states))
        
        for j in range(num_states):
            for k in range(num_states):
                min_cost = float('inf')
                for u in range(num_states):
                    cost = calculate_cost(j + 1, k + 1, u + 1)
                    
                    expected_value = 0
                    for l in range(num_states):
                        expected_value += P[k, l] * V_prev[u, l]
                    
                    value = cost + discount_factor * expected_value
                    min_cost = min(min_cost, value)
                
                V[j, k] = min_cost
                V_lo[j, k] += (discount_factor / (1 - discount_factor)) * (V[j, k] - V_prev[j, k])
                V_up[j, k] += (discount_factor / (1 - discount_factor)) * (V[j, k] - V_prev[j, k])
                V_delta[j, k] = abs(V[j, k] - V_prev[j, k])
        
        if np.max(V_delta) < tolerance:
            break
    
    return V, V_lo, V_up

def get_optimal_policy(V, P, discount_factor):
    num_states = V.shape[0]
    policy = np.zeros((num_states, num_states), dtype=np.int)
    
    for j in range(num_states):
        for k in range(num_states):
            min_cost = float('inf')
            best_action = None
            
            for u in range(num_states):
                cost = calculate_cost(j + 1, k + 1, u + 1)
                
                expected_value = 0
                for l in range(num_states):
                    expected_value += P[k, l] * V[u, l]
                
                if cost + discount_factor * expected_value < min_cost:
                    min_cost = cost + discount_factor * expected_value
                    best_action = u
            
            policy[j, k] = best_action
    
    return policy

def print_optimal_policy(policy):
    num_states = policy.shape[0]
    print("\nOptimal Policy:")
    for j in range(num_states):
        policy_row = " ".join(str(action + 1) for action in policy[j])
        print(policy_row)


# Transition matrix
P = np.array([
    [0.1, 0.3, 0.3, 0.3],
    [0.0, 0.5, 0.5, 0.0],
    [0.0, 0.0, 0.8, 0.2],
    [0.4, 0.0, 0.0, 0.6]
])

discount_factor = 0.95

# Perform value iteration
V, V_lo, V_up = value_iteration(P, discount_factor)

# Calculate average reward per step
average_reward = np.mean(V)

# Calculate total cost
total_cost = np.sum(V)

# Calculate and print the optimal policy
policy = get_optimal_policy(V, P, discount_factor)
print_optimal_policy(policy)

# Print the results
print("\nOptimal Value Function (v*):")
print(V)
print("\nLower Bounds (v_lo):")
print(V_lo)
print("\nUpper Bounds (v_up):")
print(V_up)
print("\nAverage Reward per Step:", average_reward)
print("\nTotal Cost:", total_cost)



Optimal Policy:
1 4 2 2
2 2 2 2
3 3 2 3
4 4 4 2

Optimal Value Function (v*):
[[186.33667797 234.48030831 169.46946549 187.75090855]
 [ 99.29708312 158.09141942  69.46946549  87.75090855]
 [140.85459985 153.32951466 169.46946549 124.47615589]
 [148.28419178 134.48030831 148.63613215 187.75090855]]

Lower Bounds (v_lo):
[[3540.39688139 4455.12585791 3219.91984422 3567.26726254]
 [1886.64457937 3003.73696902 1319.91984422 1667.26726254]
 [2676.23739709 2913.26077855 3219.91984422 2365.04696192]
 [2817.39964383 2555.12585791 2824.08651089 3567.26726254]]

Upper Bounds (v_up):
[[3540.39688139 4455.12585791 3219.91984422 3567.26726254]
 [1886.64457937 3003.73696902 1319.91984422 1667.26726254]
 [2676.23739709 2913.26077855 3219.91984422 2365.04696192]
 [2817.39964383 2555.12585791 2824.08651089 3567.26726254]]

Average Reward per Step: 149.99546959917066

Total Cost: 2399.9275135867306


C:\Users\Ratan\AppData\Local\Temp\ipykernel_18956\2226667210.py:60: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  policy = np.zeros((num_states, num_states), dtype=np.int)


### Method 2: Obtain optimal policy and optimal value function by policy iteration

In [36]:
import numpy as np

def calculate_cost(j, k, u):
    if k == 1 and u == j:
        return 0
    elif k > 1 and u == j == 1:
        return 200
    elif k == u == j > 1:
        return 50
    elif k > 1 and u != j and u != k:
        return 100
    elif k == 1 and u != j:
        return 300
    elif k > 1 and u == 1 and j != 1:
        return 500
    elif k > 1 and u == k and j != k:
        return 350
    elif k > 1 and (u == 1 or u == k) and j != u:
        return 400
    else:
        return 0  # or a value that makes sense in your context

def policy_iteration(P, discount_factor, max_iterations=1000):
    num_states = P.shape[0]
    num_actions = P.shape[1]
    
    # Initialize the policy and value functions
    policy = np.zeros((num_states, num_states), dtype=np.int)
    value_function = np.zeros((num_states, num_states))
    
    for _ in range(max_iterations):
        # Policy evaluation
        while True:
            new_value_function = np.copy(value_function)
            
            for j in range(num_states):
                for k in range(num_states):
                    u = policy[j, k]
                    cost = calculate_cost(j + 1, k + 1, u + 1)
                    
                    expected_value = 0
                    for l in range(num_states):
                        expected_value += P[k, l] * value_function[u, l]
                    
                    new_value_function[j, k] = cost + discount_factor * expected_value
            
            if np.max(np.abs(value_function - new_value_function)) < 1e-6:
                break
            
            value_function = new_value_function
        
        # Policy improvement
        policy_stable = True
        
        for j in range(num_states):
            for k in range(num_states):
                current_action = policy[j, k]
                min_cost = float('inf')
                best_action = None
                
                for u in range(num_actions):
                    cost = calculate_cost(j + 1, k + 1, u + 1)
                    
                    expected_value = 0
                    for l in range(num_states):
                        expected_value += P[k, l] * value_function[u, l]
                    
                    if cost + discount_factor * expected_value < min_cost:
                        min_cost = cost + discount_factor * expected_value
                        best_action = u
                
                if best_action != current_action:
                    policy_stable = False
                    policy[j, k] = best_action
        
        if policy_stable:
            break
    
    return policy, value_function

# Transition matrix
P = np.array([
    [0.1, 0.3, 0.3, 0.3],
    [0.0, 0.5, 0.5, 0.0],
    [0.0, 0.0, 0.8, 0.2],
    [0.4, 0.0, 0.0, 0.6]
])

discount_factor = 0.95

policy, value_function = policy_iteration(P, discount_factor)

# Calculate average reward per step
average_reward = np.mean(value_function)

# Calculate total cost
total_cost = np.sum(value_function)

print("Optimal Policy:")
print(policy)

print("\nOptimal Value Function:")
print(value_function)

print("\nAverage Reward per Step:", average_reward)
print("\nTotal Cost:", total_cost)

Optimal Policy:
[[0 3 1 1]
 [1 1 1 1]
 [2 2 1 2]
 [3 3 3 1]]

Optimal Value Function:
[[186.33669616 234.4803265  169.46948368 187.75092675]
 [ 99.29710132 158.09143762  69.46948368  87.75092675]
 [140.85461954 153.32953285 169.46948368 124.47617746]
 [148.28420997 134.4803265  148.63615035 187.75092675]]

Average Reward per Step: 149.99548809740222

Total Cost: 2399.9278095584355


C:\Users\Ratan\AppData\Local\Temp\ipykernel_18956\4229978039.py:28: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  policy = np.zeros((num_states, num_states), dtype=np.int)


### Method 3: Obtain optimal policy and optimal value function by linear programming. 

In [49]:
from pulp import *
import numpy as np

# Define the step-wise cost function c(j, k, u)
def c(j, k, u):
    if k == 1 and u == j:
        return 0
    elif k > 1 and u == j == 1:
        return 200
    elif k == u == j > 1:
        return 50
    elif k > 1 and u != j and u != k:
        return 100
    elif k == 1 and u != j:
        return 300
    elif k > 1 and u == 1 and j != 1:
        return 500
    elif k > 1 and u == k and j != k:
        return 350
    elif k > 1 and (u == 1 or u == k) and j != u:
        return 400
    else:
        return 0  # Return 0 for any other combination of j, k, u

# Define the discount factor α
α = 0.95

# Define the transition probabilities p_kl
p = [[0.1, 0.3, 0.3, 0.3],
     [0.0, 0.5, 0.5, 0.0],
     [0.0, 0.0, 0.8, 0.2],
     [0.4, 0.0, 0.0, 0.6]]

# Define the state and control spaces
states = list(range(1, len(p) + 1))
controls = list(range(1, len(p[0]) + 1))

# Create the LP problem
prob = LpProblem("OptimalValueFunction", LpMaximize)

# Define the decision variables
v = LpVariable.dicts("v", (states, states), lowBound=0)

# Set the objective function
prob += lpSum([v[j][k] for j in states for k in states])

# Add the constraints
for j in states:
    for k in states:
        for u in controls:
            prob += v[j][k] <= c(j, k, u) + α * lpSum([p[k - 1][l - 1] * v[u][l] for l in states])

# Solve the LP problem
prob.solve()

# Calculate evaluation metrics
total_rewards = value(prob.objective)
average_cost_per_step = total_rewards / (len(states) ** 2)

# Create arrays for optimal value function and optimal policy
optimal_value_function = np.zeros((len(states), len(states)))
optimal_policy = np.zeros((len(states), len(states)), dtype=int)

# Store optimal value function and optimal policy
for j in states:
    for k in states:
        optimal_value_function[j - 1][k - 1] = value(v[j][k])
        max_action = max(controls, key=lambda u: c(j, k, u) + α * sum([p[k - 1][l - 1] * value(v[u][l]) for l in states]))
        optimal_policy[j - 1][k - 1] = max_action

# Print the optimal value function as array
print("Optimal Value Function (v*):")
print(optimal_value_function)

# Print the optimal policy as array
print("\nOptimal Policy:")
print(optimal_policy)

# Print the evaluation metrics
print("\nAverage Cost per Step: ", average_cost_per_step)
print("\nTotal Rewards: ", total_rewards)

Optimal Value Function (v*):
[[186.3367   234.48033  169.46948  187.75093 ]
 [ 99.297101 158.09144   69.469484  87.750927]
 [140.85462  153.32953  169.46948  124.47617 ]
 [148.28421  134.48033  148.63615  187.75093 ]]

Optimal Policy:
[[4 2 3 4]
 [1 1 3 4]
 [1 2 1 4]
 [1 2 3 1]]

Average Cost per Step:  149.99548825

Total Rewards:  2399.927812
